# Derivatives of feed-forward neural network

In [1]:
from FFANN import FFANN,linearActivation
import numpy as np

In [2]:
lin=linearActivation()

In [3]:
brain = FFANN(5,3,[3,2,30,15,8,60],[lin,lin,lin,lin,lin,lin,lin])

In [4]:
#sets input to [1,2], calls feedForward and returns the output signals
x=[1,2,.1,-2,np.pi]
brain(x)

[12.01636838722204, -6.740213646203519, -45.17471351856925]

Derivative at x

In [5]:
brain.calcTotaDerivative()
brain.totalDerivatives

[[12.13473342574206,
  25.723021999055415,
  -6.632492933308833,
  -33.40786845008788,
  -48.22910753660451],
 [1.087696546293355,
  -6.99484255719055,
  3.7464473529013245,
  5.00165376935387,
  0.5292419999197451],
 [-7.626804863188536,
  -19.7163037030294,
  5.825115186723341,
  24.048560374274725,
  32.16413223913793]]

In [6]:
brain.totalNumericalDerivative()

[[12.134733425748667,
  25.723021999051337,
  -6.6324929333116955,
  -33.40786845007582,
  -48.22910753660335],
 [1.0876965462922605,
  -6.994842557184189,
  3.7464473529121145,
  5.001653769352614,
  0.5292419999183529],
 [-7.62680486318601,
  -19.716303703021982,
  5.825115186727135,
  24.048560374280232,
  32.16413223914059]]

Derivative at x=[0,1]

In [7]:
brain.calcTotaDerivative([0,1])
brain.totalDerivatives

[[12.13473342574206,
  25.723021999055415,
  -6.632492933308833,
  -33.40786845008788,
  -48.22910753660451],
 [1.087696546293355,
  -6.99484255719055,
  3.7464473529013245,
  5.00165376935387,
  0.5292419999197451],
 [-7.626804863188536,
  -19.7163037030294,
  5.825115186723341,
  24.048560374274725,
  32.16413223913793]]

In [8]:
brain.totalNumericalDerivative([0,1])

[[12.134733425741118,
  25.723021999056073,
  -6.632492933315732,
  -33.40786845008795,
  -48.22910753660676],
 [1.0876965463114674,
  -6.994842557194403,
  3.7464473529111055,
  5.001653769353465,
  0.5292419999184867],
 [-7.626804863193115,
  -19.716303703031457,
  5.825115186746514,
  24.048560374277272,
  32.16413223913844]]